## Imports

In [3]:
import pandas as pd

## Leitura da base

### Primeira leitura para checagem de dados

In [4]:
df_base_carts = pd.read_parquet('C:/Users/candi/OneDrive/Documentos/Enjoei/Case_enjoei/datasets/bronze/carts.parquet')

In [5]:
df_base_carts

,id,userId,date,products,__v
0,1,1,2020-03-02T00:00:00.000Z,"[{'productId': 1, 'quantity': 4}, {'productId'...",0
1,2,1,2020-01-02T00:00:00.000Z,"[{'productId': 2, 'quantity': 4}, {'productId'...",0
2,3,2,2020-03-01T00:00:00.000Z,"[{'productId': 1, 'quantity': 2}, {'productId'...",0
3,4,3,2020-01-01T00:00:00.000Z,"[{'productId': 1, 'quantity': 4}]",0
4,5,3,2020-03-01T00:00:00.000Z,"[{'productId': 7, 'quantity': 1}, {'productId'...",0
5,6,4,2020-03-01T00:00:00.000Z,"[{'productId': 10, 'quantity': 2}, {'productId...",0
6,7,8,2020-03-01T00:00:00.000Z,"[{'productId': 18, 'quantity': 1}]",0


# Tratamentos da base

##### O método explode() transforma cada elemento de uma lista em uma linha separada.
o código abaixo já mantém automaticamente o "userid", pois o pandas preserva as outras colunas durante o explode.

In [6]:
# explodir a lista de produtos para criar linhas separadas para cada produto
df_exploded = df_base_carts.explode('products')

In [7]:
# extraindo productId e quantity de cada dicionário

'''São Criadas  novas colunas chamadas 'productId' e 'quantity' O apply() executa a função lambda para cada linha.
A função lambda lambda x: x['productId'] pega cada dicionário (x) e extrai o valor da chave 'productId' '''

df_exploded['productId'] = df_exploded['products'].apply(lambda x: x['productId'])
df_exploded['quantity'] = df_exploded['products'].apply(lambda x: x['quantity'])

### Remoção de colunas

In [8]:
# Removemos a coluna original de "products" e a "__v" por não haver necessidade.
df_exploded = df_exploded.drop('products', axis=1)
df_exploded = df_exploded.drop('__v', axis=1)

### Conferência dos dados

In [9]:
df_exploded

,id,userId,date,productId,quantity
0,1,1,2020-03-02T00:00:00.000Z,1,4
0,1,1,2020-03-02T00:00:00.000Z,2,1
0,1,1,2020-03-02T00:00:00.000Z,3,6
1,2,1,2020-01-02T00:00:00.000Z,2,4
1,2,1,2020-01-02T00:00:00.000Z,1,10
1,2,1,2020-01-02T00:00:00.000Z,5,2
2,3,2,2020-03-01T00:00:00.000Z,1,2
2,3,2,2020-03-01T00:00:00.000Z,9,1
3,4,3,2020-01-01T00:00:00.000Z,1,4
4,5,3,2020-03-01T00:00:00.000Z,7,1


## Tratamento de datas

### Para tratar datas nesse formato (ISO 8601 com timezone), existem algumas abordagens usando pandas.

In [19]:
# Converte a coluna para datetime e extrai só a data
df_exploded['date'] = pd.to_datetime(df_exploded['date']).dt.date

In [20]:
df_exploded

,id,userId,date,productId,quantity
0,1,1,2020-03-02,1,4
0,1,1,2020-03-02,2,1
0,1,1,2020-03-02,3,6
1,2,1,2020-01-02,2,4
1,2,1,2020-01-02,1,10
1,2,1,2020-01-02,5,2
2,3,2,2020-03-01,1,2
2,3,2,2020-03-01,9,1
3,4,3,2020-01-01,1,4
4,5,3,2020-03-01,7,1


## Visualização dos dados nulos

In [21]:
# Soma de todos os valores nulos por coluna
df_exploded.isnull().sum()

id           0
userId       0
date         0
productId    0
quantity     0
dtype: int64

### No caso dessa base, não houve a presença de dados nulos

In [ ]:
'''
    Em casos da existência de dados nulos ou faltantes, poderiamos usar os metodos citados abaixo
    para tratar os casos, com base nas regras que foram impostas.
'''
# df_limpo = df_exploted.dropna()  '''Remove todas as linhas que contêm qualquer valor nulo, útil quando se tem poucas linhas com dados faltantes.'''
# df_preenchido = df_exploted.fillna('VALOR_FALTANTE')  '''Substitui todos os valores nulos por um valor específico, interessante quando se quer dados faltantes'''

## Salvando na camada prata

In [23]:
df_exploded.to_parquet('C:/Users/candi/OneDrive/Documentos/Enjoei/Case_enjoei/datasets/silver/carts.parquet', index=False)